## CNN 모델
* 컨볼루션 -> 풀링 -> 컨볼루션 -> 드롭아웃 -> 풀링 -> 신경망 -> 드롭아웃 -> 신경망

In [1]:
# 필수 라이브러리 Import
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available() # 계산을 가속해주는 CUDA를 사용할 수 있는지 확인
DEVICE = torch.device("cuda" if USE_CUDA else "cpu") # USE_CUDA가 True이면 gpu, False이면 cpu로 보내도록 가리키는 역할

In [5]:
# 에폭과 배치크기를 정함
EPOCHS     = 40
BATCH_SIZE = 64

## 데이터셋 불러오기

In [6]:
# Fashion MNIST 데이터셋 불러오기
# 60,000개의 학습 예제와 10,000개의 테스트 예제
# 흑백(grayscale)의 28x28 이미지와 10개 분류(class) 중 하나인 정답(label)으로 구성


# Train data loader
train_loader = torch.utils.data.DataLoader( # DataLoader : 데이터셋을 batch 단위로 쪼개서 학습할 때 모델의 입력으로 주는 클래스
    datasets.MNIST('./.data', #학습/테스트 데이터가 저장되는 경로
                   train=True, # 학습용 또는 테스트용 데이터셋 여부를 지정
                   download=True, #  root 에 데이터가 없는 경우 인터넷에서 다운로드
                   transform=transforms.Compose([   # torchvision의 transform package를 사용하면 DataLoader에 직접 torchvistion dataset을 넣어주고 간결하게 재구성이 가능함 (입력 변환 가능)
                       # 학습을 하려면 정규화(normalize)된 텐서 형태의 특징(feature)이 필요
                       transforms.ToTensor(), # 이미지를 텐서로 변환
                       transforms.Normalize((0.1307,), (0.3081,)) # 이미지 정규화
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

# Test data loader
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=False, # 테스트 데이터
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./.data\MNIST\raw\train-images-idx3-ubyte.gz to ./.data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./.data\MNIST\raw\train-labels-idx1-ubyte.gz to ./.data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./.data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./.data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./.data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./.data\MNIST\raw



* ToTensor() : PIL Image나 NumPy ndarray 를 FloatTensor로 변환하고, 이미지의 픽셀의 크기 값을 [0., 1.] 범위로 비례하여 조정(scale)

## 뉴럴넷으로 Fashion MNIST 학습하기

* CNN 클래스를 구현하는 init함수와 실제 데이터가 지나가는 forward 함수로 나뉨

In [7]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        # 필터 : 5x5 , 숫자를 하나만 지정해도 정사각형으로 간주함
        # 첫번째 파라미터 : 입력 채널 수 (in_channels) => 흑백 이미지이므로 1
        # 두번째 파라미터 : 출력 채널 수 (out_channel) => 필터의 개수가 10개 => 10개의 특징맵을 생성
        
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        # 입력 채널 수 : 10 (conv1의 결과물)
        # 출력 채널 수 : 20 => 필터의 개수가 20개 => 20개의 특징맵을 생성
        
        self.conv2_drop = nn.Dropout2d()
        # Dropout 적용 (Default: 0.5)
        # 모델에 규제를 주어서 overfitting을 방지
        # 무작위로 일부 뉴런을 생략시킴
        
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        # 일반 신경망을 거치면서 이전 출력 크기인 320에서 50,10순으로 작아지도록 함
        # 50은 중간값으로 임의로 지정한 것이고, 10은 분류해야 할 클래스의 개수 (10개)를 의미

    # 실제 데이터가 지나가는 길을 나타내는 함수    
    def forward(self, x):
        # 각 레이어는 conv-max pooling - ReLU를 하나의 묶음으로 간주
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) # max pooling 2x2 kernal을 의미            
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2)) # 중간에 dropout 진행
        
        x = x.view(-1, 320) # 컨볼루션 계층 2개를 거쳐 특징맵이 된 x를 FC layer에 넣기 위해서 2차원에서 1차원으로 펴기 (-1은 남는 차원 모두, 320은 x가 가진 원소개수)
        x = F.relu(self.fc1(x)) 
        x = F.dropout(x, training=self.training) # ReLU 활성화 함수를 거친 뒤 드롭아웃을 사용
        x = self.fc2(x) # 0부터 9까지 레이블을 갖는 10개의 출력값 생성
        return x

## 하이퍼파라미터
to() 함수는 모델의 파라미터들을 지정한 곳으로 보내는 역할을 합니다. 일반적으로 CPU 1개만 사용할 경우 필요는 없지만, GPU를 사용하고자 하는 경우 to("cuda")로 지정하여 GPU로 보내야 합니다. 지정하지 않을 경우 계속 CPU에 남아 있게 되며 빠른 훈련의 이점을 누리실 수 없습니다.

최적화 알고리즘으로 파이토치에 내장되어 있는 optim.SGD를 사용하겠습니다.

In [8]:
model     = Net().to(DEVICE) # 방금 만든 Net 모델 (CNN)의 인스턴스 생성
# to() 함수 : 모델의 파라미터들을 지정한 장치의 메모리로 보내는 역할
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5) # 최적화 알고리즘으로 파이토치에 내장되어 있는 optim.SGD(확률적 경사하강법)를 사용

## 학습하기

In [9]:
# 훈련 코드
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        # 학습데이터를 device의 메모리로 보냄
        
        optimizer.zero_grad() 
        # 한 번 학습이 완료 되면 기울기를 새로 계산하므로 0을 만들어 줘야됨
        
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()# backpropagation을 통해 gradient 계산 => 각 매개변수에 대한 기울기 저장
        optimizer.step() # 역전파 단계에서 수집된 기울기로 가중치 수정

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

## 테스트하기

In [10]:
def evaluate(model, test_loader):
    model.eval() # 해당 모델의 모든 레이어가 evalution mode에 들어가게 함 => 학습할 때만 필요한 Dropout, Batchnorm등의 기능을 비활성화 시킴
    test_loss = 0
    correct = 0 # 정답개수
    with torch.no_grad(): # autograde(gradient를 계산해주는 것)를 비활성화 시킴 => 필요한 메모리가 줄어들고 연산 속도 증가
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item() # 미니 배치의 오차 평균 대신 합을 받아옴

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1] 
            # 출력되는 max 값은 값과 인덱스 => 레이블에 해당하는 인덱스가 필요하므로 두번쨰 원소를 불러와야됨 => 1
            
            correct += pred.eq(target.view_as(pred)).sum().item()
            # target.view_as(pred) : target을 pred의 shape에 맞춰 비교
            # target과 pred가 일치하면 correct에 1을 더함

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 코드 돌려보기

In [11]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.343865
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.213240
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.311867
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.361549
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.570619
[1] Test Loss: 0.1668, Accuracy: 94.88%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.347148
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.564691
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.547083
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.426779
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.250864
[2] Test Loss: 0.1021, Accuracy: 96.87%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.352621
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.211621
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.479466
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.353841
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.262848
[3] Test Loss: 0.0866, Accuracy: 97.35%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.331998
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.362118
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.350231


* 정확도가 99%정도 됨